In [1]:
import matplotlib.pyplot as plt
try:
    xrange = xrange
except:
    xrange = range
from PIL import ImageGrab
import cv2
import time
import math
import datetime
import random
import io
from lib.getkeys import key_check
from lib.reinforcement import Qnetwork,updateTarget,updateTargetGraph
from lib.SQL import SQLCalls
from sys import stdout
import sqlite3
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import os
from PIL import Image
from keras import backend as K
from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Conv2D, MaxPooling2D,concatenate, Flatten, GlobalAveragePooling2D
from keras.utils import to_categorical
SQL=SQLCalls()

Using TensorFlow backend.


In [ ]:
from PIL import Image
def process_img(original_image):
    processed_img= cv2.resize(original_image,(580,580))
    return np.array(np.transpose(processed_img,(0,1,2)))
print_screen = np.array(ImageGrab.grab(bbox=(0,60,580,530)))
print(print_screen.shape)
x=process_img(print_screen)
Image.fromarray(x,'RGB')

In [ ]:

epoch=0
frame_count=0
ACTION,WAIT,DEATH,GENERATION_OVER,RESTORE=0,1,2,3,4
print("Taking picture of the top-left of the screen.")
print("Please check image to ensure it only displays the emulator.")
img=ImageGrab.grab(bbox=(0,60,580,530))
img.save("../Test.png")

#Hyper Params
update_freq = 4 #How often to perform a training step.
y = .1 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action
anneling_steps = 10000. #How many steps of training to reduce startE to endE.
pre_train_steps = 10000 #How many steps of random actions before training begins.
max_epLength = 50 #The max allowed length of our episode.
load_model = False #Whether to load a saved model.
path = "./dqn" #The path to save our model to.
h_size = 1024 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
tau = 0.001 #Rate to update target network toward primary network
img_size=84 #Size of the image.

In [2]:
Genomes=SQL.GatherGenomes()
POPULATION=len(Genomes)
#print(POPULATION)
#print(Genomes[0])
batch_size = POPULATION//4 #How many experiences to use for each training step.
BoxRadius=6
BoxLength=BoxRadius*2+1
BoxArea=(BoxLength)
gene_image=np.empty([len(Genomes),BoxLength,BoxLength,12])
gene_image.fill(0)
BUTTON_AMOUNT=6
for Genome_Num,Genome in enumerate(Genomes):
    for gene in Genome:
        genome_type=0
        if gene[0]>BoxLength:
            pass
            #print("Normal Input")
        if gene[1]>BoxLength:
            pass
            #print("Inverse Input")
            genome_type+=BUTTON_AMOUNT
        genome_type+=int(gene[1]-1000001) 
        if genome_type>=0:
            # print X,Y,Type(Type of Input,Button Pressed)
            gene_image[Genome_Num][int(gene[0]%(BoxArea)//BoxArea)][int(gene[0]%(BoxArea)%13)][genome_type]=gene[2]             
'''
[[    339 1000003       9]
 [    258 1000004       7]
 [    178 1000004       8]
'''
for l in range(100):
    for i in range(len(gene_image[l])):
        for j in range(len(gene_image[l][i])):
            for k in range(len(gene_image[l][i][j])):
                if gene_image[l][i][j][k]!=0:
                    print(l,i,j,k)
                    print(gene_image[l][i][j][k])

for j in range(1):
    print(str(datetime.datetime.now().time()))            

0 0 1 8
-1.33640552995
0 0 9 9
-1.94006164739
0 0 11 9
1.43577379681
1 0 2 7
1.21042512284
1 0 8 6
-0.591509750664
2 0 3 6
-0.935148167364
3 0 4 6
-1.17856379894
3 0 6 6
1.79662465285
4 0 7 10
-0.0324106570635
4 0 8 10
-1.41001617481
4 0 10 11
1.00619525742
5 0 6 9
-1.28843043306
6 0 1 9
-0.175603503525
6 0 6 10
-0.616046632282
6 0 8 6
-1.58128604999
7 0 2 9
1.37217322306
7 0 10 11
0.290353099155
8 0 1 8
-1.54490798669
8 0 2 7
0.400952177496
8 0 6 11
-1.15768913846
8 0 10 8
-1.99353007599
9 0 5 11
0.783043916135
10 0 10 9
-1.09079256569
10 0 12 9
0.512283700064
11 0 1 7
0.338450270089
11 0 10 11
-1.02853480636
12 0 1 11
0.97726371044
12 0 5 10
-0.776696066164
13 0 0 11
1.4933927427
13 0 1 8
-1.73705252235
13 0 1 9
0.951994384594
14 0 1 6
-0.483474227119
14 0 10 11
0.204657124546
15 0 2 7
0.211249122593
16 0 9 8
-1.49412518693
17 0 3 11
-1.42075869015
17 0 9 9
-1.45384075442
18 0 12 11
1.15854365673
19 0 4 8
-0.654377880184
19 0 12 8
1.7265541551
20 0 5 6
-1.03561510056
20 0 11 7
1.7811

In [3]:
def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())
def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [4]:
def setup_genomes():
    BoxRadius=6
    BoxLength=BoxRadius*2+1
    BoxArea=(BoxLength)
    gene_image=np.empty([len(Genomes),BoxLength,BoxLength,12])
    gene_image.fill(0)
    BUTTON_AMOUNT=6
    for Genome_Num,Genome in enumerate(Genomes):
        for gene in Genome:
            genome_type=0
            if gene[0]>BoxLength:
                pass
                #print("Normal Input")
            if gene[1]>BoxLength:
                pass
                #print("Inverse Input")
                genome_type+=BUTTON_AMOUNT
            genome_type+=int(gene[1]-1000001) 
            if genome_type>=0:
                # print X,Y,Type(Type of Input,Button Pressed)
                gene_image[Genome_Num][int(gene[0]%(BoxArea)//BoxArea)][int(gene[0]%(BoxArea)%13)][genome_type]=gene[2] 
    return gene_image

In [5]:
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

con=sqlite3.connect('DQN.db', detect_types=sqlite3.PARSE_DECLTYPES,isolation_level=None)
cur = con.cursor()
cur.execute("PRAGMA synchronous = OFF;")
cur.execute("PRAGMA journal_mode=WAL;")
cur.execute("PRAGMA read_uncommitted = true;")
cur.execute("SELECT GenomeNum,Gene,GeneContent FROM Genes ORDER BY Genome,Gene")
# trainBatch=SQL.gain_history()
# batch_size=20
# trainBatch=trainBatch[0:batch_size]
# time_stamp=datetime.datetime.now().time()
# genomeImages=setup_genomes()
# genomeImages=genomeImages[0:batch_size]
# for i in range(len(genomeImages)):
#     sql = ''' INSERT INTO example_genes
#                (GenomeKey,GeneImage)
#                VALUES
#                (?,?)'''
#     cur.execute(sql, (str(time_stamp)+str(i),genomeImages[i]))
# con.commit()    
# [0] Image
# [1] GenomeNUm
# [2] Score
# [3] ImageEn d
# states=trainBatch[:,0]
# GenomeNum=trainBatch[:,1]
# score=trainBatch[:,2]
# states_after=trainBatch[:,3]
# for i in range(len(trainBatch)):
#     sql=''' INSERT INTO example_images
#                (GenomeKey,Score,Image,ImageEnd)
#                VALUES
#                (?,?,?,?)'''
#     cur.execute(sql, (states[i],GenomeNum[i],score[i],states_after[i]))
# con.commit()  
# sql=''' INSERT INTO example_timestamps
#                 (timestamp)
#                 VALUES
#                 (?)'''
# cur.execute(sql, (str(time_stamp),))
# con.commit()    


In [6]:
sql = '''Select image,score,imageEnd,geneImage
        from example_images ei
        inner join 
        example_genes eg on ei.genomeKey==eg.genomeKey
        where score=-1 LIMIT 50'''
cur.execute(sql)
results=cur.fetchall()
results=np.array(results)
sql = '''Select image,score,imageEnd,geneImage
        from example_images ei
        inner join 
        example_genes eg on ei.genomeKey==eg.genomeKey
        where score=0 LIMIT 50'''
cur.execute(sql)
results2=cur.fetchall()
results2=np.array(results2)

In [8]:
def make_model():
    image_model_inputs = Input(shape=X[0].shape,dtype='float32',name='main_image')
    image_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(image_model_inputs)
    image_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(image_model)

    image_model=Conv2D(32, (1,1),strides=2, padding='valid', activation='relu')(image_model)
    image_model=Conv2D(32, (1,1),strides=2, padding='valid', activation='relu')(image_model)

    image_model=Conv2D(8, (1,1),strides=3, padding='same', activation='relu')(image_model)
    image_model=Conv2D(8, (1,1),strides=2, padding='same', activation='relu')(image_model)
    
    image_model=Flatten()(image_model)
    
    gene_model_inputs = Input(shape=X_gene[0].shape,dtype='float32',name='gene_image')
    gene_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(gene_model_inputs)
    gene_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(gene_model)
    
    gene_model=Conv2D(8, (1,1),strides=3, padding='same', activation='relu')(gene_model)
    gene_model=Conv2D(8, (1,1),strides=2, padding='same', activation='relu')(gene_model)
    
    gene_model=Flatten()(gene_model)
    
    image_model=concatenate([image_model,gene_model])
    
    image_model=Dense(16, activation='relu')(image_model)
    image_model=Dense(8, activation='relu')(image_model)
    image_model_preditions=Dense(2, activation='softmax')(image_model)
    model=Model(inputs=[image_model_inputs,gene_model_inputs],outputs=image_model_preditions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
K.clear_session()
X=np.reshape(np.vstack(np.concatenate((results[:,0],results2[:,0]))),(-1,580,580,3))
np.random.shuffle(X)
X_gene=np.reshape(np.vstack(np.concatenate((results[:,3],results2[:,3]))),(-1,13,13,12))
np.random.shuffle(X_gene)
Y=np.vstack(np.concatenate((results[:,1],results2[:,1])))
Y = to_categorical(Y, num_classes=2)
np.random.shuffle(Y)
y_train=Y[0:90]
y_test=Y[90:100]
x_train=X[0:90]
x_test=X[90:100]
x_gene_train=X_gene[0:90]
x_gene_test=X_gene[90:100]
model = make_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_image (InputLayer)          (None, 580, 580, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 579, 579, 16)  208         main_image[0][0]                 
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 578, 578, 16)  1040        conv2d_1[0][0]                   
____________________________________________________________________________________________________
gene_image (InputLayer)          (None, 13, 13, 12)    0                                            
___________________________________________________________________________________________

In [9]:
epochs=60
batch_size=16
history = model.fit([x_train,x_gene_train], y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=([x_test,x_gene_test], y_test))

#model1.evaluate(test_data, test_labels_one_hot)

Train on 90 samples, validate on 10 samples
Epoch 1/60
90/90 [==============================] - 5s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/60
90/90 [==============================] - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_

In [59]:
model.evaluate(x_test, y_test)

20/20 [==============================] - 0s


[0.27957120537757874, 1.0]

In [81]:
def make_model():
    model = Sequential()
    model.add(Conv2D(16, (2, 2), padding='valid', activation='relu', input_shape=X[0].shape))
    model.add(Conv2D(16, (2, 2), padding='valid', activation='relu'))
    #model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(8, (1, 1),strides=3, padding='same', activation='relu'))
    model.add(Conv2D(8, (1, 1),strides=2, activation='relu'))
    #model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
K.clear_session()
X=np.reshape(np.vstack(np.concatenate((results[:,3],results2[:,3]))),(-1,13,13,12))
np.random.shuffle(X)
Y=np.vstack(np.concatenate((results[:,1],results2[:,1])))
Y = to_categorical(Y, num_classes=2)
np.random.shuffle(Y)
y_train=Y[0:130]
y_test=Y[130:150]
x_train=X[0:130]
x_test=X[130:150]
model = make_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 12, 12, 16)        784       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        1040      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 8)           136       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 8)           72        
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________